# ID2214/FID3214 Assignment 3 Group no. 11
### Project members: 
[Ivan Klabucar, klabucar@kth.se]
[Nicola Toscan, toscan@kth.se]
[Santiago Carlino, smcr@kth.se]

### Declaration
By submitting this solution, it is hereby declared that all individuals listed above have contributed to the solution, either with code that appear in the final solution below, or with code that has been evaluated and compared to the final solution, but for some reason has been excluded. It is also declared that all project members fully understand all parts of the final solution and can explain it upon request.

It is furthermore declared that the code below is a contribution by the project members only, and specifically that no part of the solution has been copied from any other source (except for lecture slides at the course ID2214/FID3214) and no part of the solution has been provided by someone not listed as project member above.

It is furthermore declared that it has been understood that no other library/package than the Python 3 standard library, NumPy, pandas, time and sklearn.tree, may be used in the solution for this assignment.

### Instructions
All parts of the assignment starting with number 1 below are mandatory. Satisfactory solutions
will give 1 point (in total). If they in addition are good (all parts work more or less 
as they should), completed on time (submitted before the deadline in Canvas) and according
to the instructions, together with satisfactory solutions of all parts of the assignment starting 
with number 2 below, then the assignment will receive 2 points (in total).

Note that you do not have to develop the code directly within the notebook
but may instead copy the comments and test cases to a more convenient development environment
and when everything works as expected, you may paste your functions into this
notebook, do a final testing (all cells should succeed) and submit the whole notebook 
(a single file) in Canvas (do not forget to fill in your group number and names above).

## Load NumPy, pandas, time and DecisionTreeClassifier from sklearn.tree

In [3]:
import numpy as np
import pandas as pd
import time
from sklearn.tree import DecisionTreeClassifier

## Reused functions from Assignment 1

In [14]:
# Copy and paste functions from Assignment 1 here that you need for this assignment

def create_column_filter(df):
    df_copy = df.copy()
    cols = [x for x in df.columns if not (x == 'CLASS' or x == 'ID')]
    not_dropped = df_copy.loc[:, cols].dropna(how='all', axis='columns').columns
    df_copy.drop(columns=[c for c in cols if not c in not_dropped], inplace=True)
    for c in df_copy:
        if c == 'CLASS' or c == 'ID': continue
        u_vals = pd.unique(df_copy[c])
        num_of_unique = len(u_vals)
        if pd.isna(u_vals).any(): num_of_unique -= 1
        if num_of_unique == 1: df_copy.drop(columns=c,inplace=True)
    return df_copy, list(df_copy.columns)

def apply_column_filter(df, column_filter):
    new_df = df.copy()
    to_drop = [c for c in new_df.columns if c not in column_filter]
    new_df.drop(columns=to_drop, inplace=True)
    return new_df

def apply_imputation(df, imputation):
    new_df = df.copy()
    for c in imputation:
        new_df[c].fillna(imputation[c], inplace=True)
    return new_df

def create_imputation(df):
    imputation = dict()
    for c in df:
        if c in ['CLASS', 'ID']: continue
        if df[c].dtype in [int, float]:
            mean = df[c].mean()
            if pd.isna(mean): mean = 0
            imputation[c] = mean
        elif df[c].dtype.name == 'category':
            mode = df[c].mode()
            if len(mode) == 0: mode = df[c].cat.categories[0]
            else: mode = mode[0]
            imputation[c] = mode
        elif df[c].dtype.name == 'object':
            mode = df[c].mode()
            if len(mode) == 0: mode = ''
            else: mode = mode[0]
            imputation[c] = mode
    
    return apply_imputation(df, imputation), imputation

def apply_one_hot(df, one_hot):
    df_new = df.copy()
    for c in df_new:
        if c in ['CLASS', 'ID']: continue
        if not df_new[c].dtype.name in ['category', 'object']: continue

        for v in one_hot[c]:
            df_new[f"{c}-{v}"] = df_new[c].map(lambda x: 1.0 if x == v else 0.0)
        df_new.drop(c, inplace=True, axis=1)
    return df_new

def create_one_hot(df):
    df_new = df.copy()
    # df_new.drop(['CLASS'], inplace=True, axis=1)
    # one_hot = { key:set(value.values()) for key, value in df_new.apply(lambda col: col.unique()).to_dict().items() }
    # return apply_one_hot(df, one_hot), one_hot
    
    one_hot = {}
    for c in df_new:
        if c in ['CLASS', 'ID']: continue
        if not df_new[c].dtype.name in ['category', 'object']: continue
        unique_values = sorted(df_new[c].unique())
        one_hot[c] = unique_values
    return apply_one_hot(df, one_hot), one_hot

def brier_score(df, correctlabels):
    nLabels = len(df.columns)
    labelMaps = [ np.zeros(nLabels) for _ in range(len(correctlabels)) ]
    for l, lm in zip(correctlabels, labelMaps):
        lm[np.where(df.columns==l)[0][0]] = 1.0

    errors = [ np.sum(np.power(labelMaps[index] - row, 2)) for index, row in df.iterrows() ]
    return sum(errors) / len(errors)

def accuracy(df, correctlabels):
    m = df.idxmax(axis="columns")
    return len([ 1 for r, label in zip(m, correctlabels) if r == label]) / len(m)

def auc(df, correctlabels):
    total_auc = 0.0

    for c in df:
        scores = []
        thresholds = list(df[c])
        corrected = [ l == c for l in correctlabels]

        for t in thresholds:
            predictions = list(df[c] >= t)
            x_y = list(zip(predictions, corrected))
            tp = len([1 for p, l in x_y if p and l])
            tn = len([1 for p, l in x_y if (not p) and (not l)])
            fp = len([1 for p, l in x_y if p and (not l)])
            fn = len([1 for p, l in x_y if (not p) and l])

            tpr = tp / (tp + fn)
            fpr = fp / (fp + tn)

            scores.append((t, tpr, fpr))
        scores =  [[0,0,0]] + sorted(scores, key=lambda x: -x[0]) + [[1, 1, 1]]

        auc = 0.0
        lastTPR = 0.0
        lastFPR = 0.0
        for t, tpr, fpr in scores:
            auc += lastTPR * (fpr - lastFPR)
            auc += (tpr - lastTPR) * (fpr - lastFPR) / 2.0
        
            lastTPR = tpr
            lastFPR = fpr
        
        total_auc += auc * (
            len([ 1 for l in correctlabels if l == c]) / len(correctlabels)
        )

    return total_auc

## 1. Define the class RandomForest

In [15]:
# Define the class RandomForest with three functions __init__, fit and predict (after the comments):
#
# Input to __init__: 
# self - the object itself
#
# Output from __init__:
# <nothing>
# 
# This function does not return anything but just initializes the following attributes of the object (self) to None:
# column_filter, imputation, one_hot, labels, model
#
# Input to fit:
# self      - the object itself
# df        - a dataframe (where the column names "CLASS" and "ID" have special meaning)
# no_trees  - no. of trees in the random forest (default = 100)
#
# Output from fit:
# <nothing>
#
# The result of applying this function should be:
#
# self.column_filter - a column filter (see Assignment 1) from df
# self.imputation    - an imputation mapping (see Assignment 1) from df
# self.one_hot       - a one-hot mapping (see Assignment 1) from df
# self.labels        - a (sorted) list of the categories of the "CLASS" column of df
# self.model         - a random forest, consisting of no_trees trees, where each tree is generated from a bootstrap sample
#                      and the number of evaluated features is log2|F| where |F| is the total number of features
#                      (for details, see lecture slides)
#
# Note that the function does not return anything but just assigns values to the attributes of the object.
#
# Hint 1: First create the column filter, imputation and one-hot mappings
#
# Hint 2: Then get the class labels and the numerical values (as an ndarray) from the dataframe after dropping the class labels 
#
# Hint 3: Generate no_trees classification trees, where each tree is generated using DecisionTreeClassifier 
#         from a bootstrap sample (see lecture slides), e.g., generated by np.random.choice (with replacement) 
#         from the row numbers of the ndarray, and where a random sample of the features are evaluated in
#         each node of each tree, of size log2(|F|), where |F| is the total number of features;
#         see the parameter max_features of DecisionTreeClassifier
#
# Input to predict:
# self - the object itself
# df   - a dataframe
# 
# Output from predict:
# predictions - a dataframe with class labels as column names and the rows corresponding to
#               predictions with estimated class probabilities for each row in df, where the class probabilities
#               are the averaged probabilities output by each decision tree in the forest
#
# Hint 1: Drop any "CLASS" and "ID" columns of the dataframe first and then apply column filter, imputation and one_hot
#
# Hint 2: Iterate over the trees in the forest to get the prediction of each tree by the method predict_proba(X) where 
#         X are the (numerical) values of the transformed dataframe; you may get the average predictions of all trees,
#         by first creating a zero-matrix with one row for each test instance and one column for each class label, 
#         to which you add the prediction of each tree on each iteration, and then finally divide the prediction matrix
#         by the number of trees.
#
# Hint 3: You may assume that each bootstrap sample that was used to generate each tree has included all possible
#         class labels and hence the prediction of each tree will contain probabilities for all class labels
#         (in the same order). Note that this assumption may be violated, and this limitation will be addressed 
#         in the next part of the assignment. 

class RandomForest:
    def __init__(self):
        self.column_filter = None
        self.imputation = None
        self.one_hot = None
        self.labels = None
        self.model = None
        # This is for 2b. performance using out-of-bag predictions
        #self.model_bsmpls = dict() # index of tree in self.model : list of indexes used for b sample
    
    def fit(self, df, no_trees=100):
        local_df, self.column_filter = create_column_filter(df)
        local_df, self.imputation = create_imputation(local_df)
        local_df, self.one_hot = create_one_hot(local_df)
        local_df["CLASS"] = local_df["CLASS"].astype("category")
        self.labels = sorted(local_df["CLASS"].cat.categories)
        local_df.dropna(how='any', axis='index', subset=['CLASS'], inplace=True)
        
        self.model = []
        size_of_bsample = len(local_df) # this way (1 - (1/n))^n is approximately 1/3, so each sample will contain 2/3 unique instances
        for i in range(no_trees):
            bsmpl_indexes = np.random.choice(local_df.index, size_of_bsample)
            #self.model_bsmpls[i] = bsmpl_indexes
            Y = local_df.loc[bsmpl_indexes, 'CLASS']
            X = local_df.loc[bsmpl_indexes].drop(['CLASS', 'ID'], axis='columns', errors='ignore')
            tree = DecisionTreeClassifier(max_features='log2')
            tree.fit(X, Y)
            self.model.append(tree)
    
    def predict(self, df):
        local_df = df.drop(['CLASS', 'ID'], axis='columns', errors='ignore')
        local_df = apply_column_filter(local_df, self.column_filter)
        local_df = apply_imputation(local_df, self.imputation)
        local_df = apply_one_hot(local_df, self.one_hot)
        
        predictions = pd.DataFrame(np.zeros((len(local_df), len(self.labels))), columns=self.labels)
        for tree in self.model:
            predictions = predictions + pd.DataFrame(tree.predict_proba(local_df), columns=self.labels)
        predictions = predictions / len(self.model) 
        return predictions


In [16]:
# Test your code (leave this part unchanged, except for if auc is undefined)

train_df = pd.read_csv("tic-tac-toe_train.csv")

test_df = pd.read_csv("tic-tac-toe_test.csv")

rf = RandomForest()

t0 = time.perf_counter()
rf.fit(train_df)
print("Training time: {:.2f} s.".format(time.perf_counter()-t0))

test_labels = test_df["CLASS"]

t0 = time.perf_counter()
predictions = rf.predict(test_df)

print("Testing time: {:.2f} s.".format(time.perf_counter()-t0))

print("Accuracy: {:.4f}".format(accuracy(predictions,test_labels)))
print("AUC: {:.4f}".format(auc(predictions,test_labels))) # Comment this out if not implemented in assignment 1
print("Brier score: {:.4f}".format(brier_score(predictions,test_labels))) # Comment this out if not implemented in assignment 1

Training time: 0.44 s.
Testing time: 0.19 s.
Accuracy: 0.9123
AUC: 0.9894
Brier score: 0.1760


In [17]:
train_labels = train_df["CLASS"]
predictions = rf.predict(train_df)
print("Accuracy on training set: {0:.4f}".format(accuracy(predictions,train_labels)))
print("AUC on training set: {0:.4f}".format(auc(predictions,train_labels))) # Comment this out if not implemented in assignment 1
print("Brier score on training set: {0:.4f}".format(brier_score(predictions,train_labels))) # Comment this out if not implemented in assignment 1

Accuracy on training set: 1.0000
AUC on training set: 1.0000
Brier score on training set: 0.0205


### Comment on assumptions, things that do not work properly, etc.


## 2a. Handling trees with non-aligned predictions

In [69]:
# Define a revised version of the class RandomForest with the same input and output as described in part 1 above,
# where the predict function is able to handle the case where the individual trees are trained on bootstrap samples
# that do not include all class labels in the original training set. This leads to that the class probabilities output
# by the individual trees in the forest do not refer to the same set of class labels.
#
# Hint 1: The categories obtained with <pandas series>.cat.categories are sorted in the same way as the class labels
#         of a DecisionTreeClassifier; the latter are obtained by <DecisionTreeClassifier>.classes_ 
#         The problem is that classes_ may not include all possible labels, and hence the individual predictions 
#         obtained by <DecisionTreeClassifier>.predict_proba may be of different length or even if they are of the same
#         length do not necessarily refer to the same class labels. You may assume that each class label that is not included
#         in a bootstrap sample should be assigned zero probability by the tree generated from the bootstrap sample. 
#
# Hint 2: Create a mapping from the complete (and sorted) set of class labels l0, ..., lk-1 to a set of indexes 0, ..., k-1,
#         where k is the number of classes
#
# Hint 3: For each tree t in the forest, create a (zero) matrix with one row per test instance and one column per class label,
#         to which one column is added at a time from the output of t.predict_proba 
#
# Hint 4: For each column output by t.predict_proba, its index i may be used to obtain its label by t.classes_[i];
#         you may then obtain the index of this label in the ordered list of all possible labels from the above mapping (hint 2); 
#         this index points to which column in the prediction matrix the output column should be added to 

class RandomForest:
    def __init__(self):
        self.column_filter = None
        self.imputation = None
        self.one_hot = None
        self.labels = None
        self.model = None

    def fit(self, df, no_trees=100):
        local_df, self.column_filter = create_column_filter(df)
        local_df, self.imputation = create_imputation(local_df)
        local_df, self.one_hot = create_one_hot(local_df)
        local_df["CLASS"] = local_df["CLASS"].astype("category")
        self.labels = sorted(local_df["CLASS"].cat.categories)
        local_df.dropna(how='any', axis='index', subset=['CLASS'], inplace=True)
        
        self.model = []
        size_of_bsample = len(local_df) # this way (1 - (1/n))^n is approximately 1/3, so each sample will contain 2/3 unique instances
        for i in range(no_trees):
            bsmpl_indexes = np.random.choice(local_df.index, size_of_bsample)
            Y = local_df.loc[bsmpl_indexes, 'CLASS']
            X = local_df.loc[bsmpl_indexes].drop(['CLASS', 'ID'], axis='columns', errors='ignore')
            tree = DecisionTreeClassifier(max_features='log2')
            tree.fit(X, Y)
            self.model.append(tree)
    
    def predict(self, df):
        local_df = df.drop(['CLASS', 'ID'], axis='columns', errors='ignore')
        local_df = apply_column_filter(local_df, self.column_filter)
        local_df = apply_imputation(local_df, self.imputation)
        local_df = apply_one_hot(local_df, self.one_hot)
        
        predictions = pd.DataFrame(np.zeros((len(local_df), len(self.labels))), columns=self.labels)
        for tree in self.model:
            curr_predictions = pd.DataFrame(tree.predict_proba(local_df))
            pp = pd.DataFrame()
            for l in self.labels:
                if l in tree.classes_:
                    pp[l] = curr_predictions[list(tree.classes_).index(l)]
                else:
                    pp[l] = [ 0.0 ] * curr_predictions.shape[0]
            predictions = predictions + pd.DataFrame(pp, columns=self.labels)
        predictions = predictions / len(self.model) 
        return predictions

In [70]:
train_df = pd.read_csv("anneal_train.csv")
test_df = pd.read_csv("anneal_test.csv")
rf = RandomForest()
rf.fit(train_df)
predictions = rf.predict(test_df)

In [71]:
# Test your code (leave this part unchanged, except for if auc is undefined)

train_df = pd.read_csv("anneal_train.csv")

test_df = pd.read_csv("anneal_test.csv")

rf = RandomForest()

t0 = time.perf_counter()
rf.fit(train_df)
print("Training time: {:.2f} s.".format(time.perf_counter()-t0))

test_labels = test_df["CLASS"]

t0 = time.perf_counter()
predictions = rf.predict(test_df)
print("Testing time: {:.2f} s.".format(time.perf_counter()-t0))

print("Accuracy: {:.4f}".format(accuracy(predictions,test_labels)))
print("AUC: {:.4f}".format(auc(predictions,test_labels))) # Comment this out if not implemented in assignment 1
print("Brier score: {:.4f}".format(brier_score(predictions,test_labels))) # Comment this out if not implemented in assignment 1

Training time: 0.52 s.
Testing time: 0.54 s.
Accuracy: 0.9465
AUC: 0.9667
Brier score: 0.1012


## 2b. Estimate predictive performance using out-of-bag predictions

In [8]:
# Define an extended version of the class RandomForest with the same input and output as described in part 2a above,
# where the results of the fit function also should include:
# self.oob_acc - the accuracy estimated on the out-of-bag predictions, i.e., the fraction of training instances for 
#                which the given (correct) label is the same as the predicted label when using only trees for which
#                the instance is out-of-bag
#
# Hint 1: You may first create a zero matrix with one row for each training instance and one column for each class label
#         and one zero vector to allow for storing aggregated out-of-bag predictions and the number of out-of-bag predictions
#         for each training instance, respectively
#
# Hint 2: After generating a tree in the forest, iterate over the indexes that were not included in the bootstrap sample
#         and add a prediction of the tree to the out-of-bag prediction matrix and update the count vector
#
# Hint 3: Note that the input to predict_proba has to be a matrix; from a single vector (row) x, a matrix with one row
#         can be obtained by x[None,:]
#
# Hint 4: Finally, divide each row in the out-of-bag prediction matrix with the corresponding element of the count vector



In [9]:
# Test your code (leave this part unchanged, except for if auc is undefined)

train_df = pd.read_csv("anneal_train.csv")

test_df = pd.read_csv("anneal_test.csv")

rf = RandomForest()

t0 = time.perf_counter()
rf.fit(train_df)
print("Training time: {:.2f} s.".format(time.perf_counter()-t0))

print("OOB accuracy: {:.4f}".format(rf.oob_acc))

test_labels = test_df["CLASS"]

t0 = time.perf_counter()
predictions = rf.predict(test_df)
print("Testing time: {:.2f} s.".format(time.perf_counter()-t0))

print("Accuracy: {:.4f}".format(accuracy(predictions,test_labels)))
print("AUC: {:.4f}".format(auc(predictions,test_labels))) # Comment this out if not implemented in assignment 1
print("Brier score: {:.4f}".format(brier_score(predictions,test_labels))) # Comment this out if not implemented in assignment 1

Training time: 1.96 s.
OOB accuracy: 0.9555
Testing time: 0.06 s.
Accuracy: 0.9488
AUC: 0.9718
Brier score: 0.0986


In [10]:
train_labels = train_df["CLASS"]
rf = RandomForest()
rf.fit(train_df)
predictions = rf.predict(train_df)
print("Accuracy on training set: {0:.2f}".format(accuracy(predictions,train_labels)))
print("AUC on training set: {0:.2f}".format(auc(predictions,train_labels)))
print("Brier score on training set: {0:.2f}".format(brier_score(predictions,train_labels)))

Accuracy on training set: 1.00
AUC on training set: 1.00
Brier score on training set: 0.01


### Comment on assumptions, things that do not work properly, etc.